In [1]:
from game import UltimateTicTacToe
from player import Player
from MCTS_player import MCTSPlayer

from tqdm import tqdm
from collections import Counter

In [ ]:
p1 = Player(1)
# p2 = Player(2)
p2 = MCTSPlayer(2, exploration_weight=0.01)
p2.train(100000)
# print(p2.explored["0"*82])
game = UltimateTicTacToe(p1, p2)
results = []
for _ in tqdm(range(100)):
    results.append(game.play_game())

print(Counter(results))

In [3]:
p1 = Player(1)
p2 = Player(2)
# p2 = MCTSPlayer(2, exploration_weight=0.01)
# p2.train(100000)
# print(p2.explored["0"*82])
game = UltimateTicTacToe(p1, p2)
results = []
for _ in tqdm(range(10000)):
    results.append(game.play_game())

print(Counter(results))

100%|██████████| 10000/10000 [00:15<00:00, 664.88it/s]

Counter({2: 3930, 1: 3877, -1: 2193})
